In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession.builder.appName("Our First Spark Example").getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,400 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,644 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-up

In [ ]:
data=[("Alice",25),("Bob",30),("Charlie",35)]
df=spark.createDataFrame(data,["name","age"])
df.show()

+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/sandeep/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
sc=spark.sparkContext
path="diabetes.csv"
df=spark.read.csv('/content/drive/My Drive/sandeep/diabetes.csv',header=True)
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
dfrdd=df.rdd
type(dfrdd)

pyspark.rdd.RDD

In [ ]:
dfrdd.take(4)

[Row(Pregnancies='6', Glucose='148', BloodPressure='72', SkinThickness='35', Insulin='0', BMI='33.6', DiabetesPedigreeFunction='0.627', Age='50', Outcome='1'),
 Row(Pregnancies='1', Glucose='85', BloodPressure='66', SkinThickness='29', Insulin='0', BMI='26.6', DiabetesPedigreeFunction='0.351', Age='31', Outcome='0'),
 Row(Pregnancies='8', Glucose='183', BloodPressure='64', SkinThickness='0', Insulin='0', BMI='23.3', DiabetesPedigreeFunction='0.672', Age='32', Outcome='1'),
 Row(Pregnancies='1', Glucose='89', BloodPressure='66', SkinThickness='23', Insulin='94', BMI='28.1', DiabetesPedigreeFunction='0.167', Age='21', Outcome='0')]

In [ ]:
dfrdd.map(lambda c:(c[0],c[1],c[2],c[3])).take(4)

[('6', '148', '72', '35'),
 ('1', '85', '66', '29'),
 ('8', '183', '64', '0'),
 ('1', '89', '66', '23')]

In [ ]:
dfrdd.map(lambda c:(c[0],c[1],c[2],c[3])).filter(lambda c:c[3]=='35').take(5)

[('6', '148', '72', '35'),
 ('0', '137', '40', '35'),
 ('9', '119', '80', '35'),
 ('6', '148', '72', '35'),
 ('0', '137', '40', '35')]

In [ ]:
df.createOrReplaceGlobalTempView("diabeticview")
from pyspark.sql import SQLContext
sqlc=SQLContext(sc)
sqlc.sql("select * from global_temp.diabeticview").show(10)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [ ]:
sqlc.sql("select Glucose,BloodPressure,SkinThickness from global_temp.diabeticview").show(10)

+-------+-------------+-------------+
|Glucose|BloodPressure|SkinThickness|
+-------+-------------+-------------+
|    148|           72|           35|
|     85|           66|           29|
|    183|           64|            0|
|     89|           66|           23|
|    137|           40|           35|
|    116|           74|            0|
|     78|           50|           32|
|    115|            0|            0|
|    197|           70|           45|
|    125|           96|            0|
+-------+-------------+-------------+
only showing top 10 rows



In [ ]:
sqlc.sql("select * from global_temp.diabeticview where Glucose>180 and BloodPressure>100").show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          0|    189|          104|           25|      0|34.3|                   0.435| 41|      1|
|          4|    189|          110|           31|      0|28.5|                    0.68| 37|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+



In [ ]:
sqlc.sql("select Outcome,count(*) as count from global_temp.diabeticview group by Outcome").show(10)

+-------+-----+
|Outcome|count|
+-------+-----+
|      0|  509|
|      1|  286|
+-------+-----+



In [ ]:
SimpleData=[("James","Sales",3000),("Miachel","Sales",4600),("Robert","Sales",4100),
            ("Maitra","Finance",3000),("James","Sales",3000),("Scott","Finance",3300),
            ("Jen","Finanace",3900),("Jeff","Marketting",3000),("Kumar","Marketting",2000),
            ("Saif","Sales",4100)]
schema=["employee_name","department","salary"]
df=spark.createDataFrame(data=SimpleData,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Miachel      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maitra       |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finanace  |3900  |
|Jeff         |Marketting|3000  |
|Kumar        |Marketting|2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [ ]:
df.createOrReplaceTempView("empview")
from pyspark.sql import SQLContext
sqlc=SQLContext(sc)
sqlc.sql("select * from empview").show(10)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Miachel|     Sales|  4600|
|       Robert|     Sales|  4100|
|       Maitra|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|  Finanace|  3900|
|         Jeff|Marketting|  3000|
|        Kumar|Marketting|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [ ]:
sqlc.sql("select department,sum(salary) from empview group by department").show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|      18800|
|   Finance|       6300|
|  Finanace|       3900|
|Marketting|       5000|
+----------+-----------+



In [ ]:
data=[(100,2,1),(200,3,4),(300,4,4)]
df=spark.createDataFrame(data).toDF("col1","col2","col3")
df.select(df.col1+df.col2).show()
df.select(df.col1-df.col2).show()
df.select(df.col1*df.col2).show()
df.select(df.col1/df.col2).show()
df.select(df.col1%df.col2).show()
df.select(df.col1>df.col2).show()
df.select(df.col1<df.col2).show()
df.select(df.col1+df.col2).show()

+-------------+
|(col1 + col2)|
+-------------+
|          102|
|          203|
|          304|
+-------------+

+-------------+
|(col1 - col2)|
+-------------+
|           98|
|          197|
|          296|
+-------------+

+-------------+
|(col1 * col2)|
+-------------+
|          200|
|          600|
|         1200|
+-------------+

+-----------------+
|    (col1 / col2)|
+-----------------+
|             50.0|
|66.66666666666667|
|             75.0|
+-----------------+

+-------------+
|(col1 % col2)|
+-------------+
|            0|
|            2|
|            0|
+-------------+

+-------------+
|(col1 > col2)|
+-------------+
|         true|
|         true|
|         true|
+-------------+

+-------------+
|(col1 < col2)|
+-------------+
|        false|
|        false|
|        false|
+-------------+

+-------------+
|(col1 + col2)|
+-------------+
|          102|
|          203|
|          304|
+-------------+

